# <center><font color=DarkRed> Partie 2: Modélisation et évaluation</font></center><br>

L'objectif général d'une méthode de recommandation est de suggérer des éléments (produits, services, contenus, etc.) à un utilisateur en fonction de ses préférences et de ses comportements passés. Les méthodes de recommandation utilisent des algorithmes pour analyser les données sur les choix et les actions de l'utilisateur, ainsi que sur les caractéristiques des éléments à recommander. En utilisant ces données, les méthodes de recommandation peuvent identifier les éléments qui sont les plus susceptibles de plaire à l'utilisateur et les lui proposer de manière personnalisée.

Le but de cette personnalisation est d'améliorer l'expérience utilisateur en offrant des recommandations pertinentes et adaptées à ses goûts, ce qui peut inciter l'utilisateur à interagir davantage avec la plateforme ou le système de recommandation. 

Les méthodes de recommandation sont utilisées dans une variété de contextes, notamment dans les systèmes de commerce électronique, les plateformes de streaming de musique ou de vidéos, les réseaux sociaux, les systèmes de recommandation de contenus éducatifs, les applications de voyage, etc.

Pour ce projet, je vais utiliser plusieurs modèles de la librairie **Implicit** et **Surprise** : 

- **Filtrage collaboratif (Collaborative Filtering)** : Cette méthode repose sur l'hypothèse que les utilisateurs ayant des préférences similaires dans le passé sont susceptibles d'avoir des préférences similaires à l'avenir. Cette méthode fonctionne en identifiant les utilisateurs qui ont des préférences similaires à l'aide de données de notation ou d'interaction, puis en recommandant des éléments qui ont été aimés par d'autres utilisateurs similaires. Les algorithmes de filtrage collaboratif peuvent être basés sur les utilisateurs (user-based) ou sur les éléments (item-based).

- **Facteurisation de matrices (Matrix Factorization)** : Cette méthode consiste à décomposer une matrice de données utilisateur-élément en deux matrices plus petites qui représentent les caractéristiques latentes des utilisateurs et des éléments. Ces matrices peuvent ensuite être utilisées pour prédire les préférences de l'utilisateur pour les éléments qu'il n'a pas encore notés ou interagis.

- **Modèles basés sur le contenu (Content-Based)** : Cette méthode utilise les caractéristiques des éléments à recommander pour identifier des éléments similaires à ceux que l'utilisateur a aimé ou interagi. Par exemple, un système de recommandation de films pourrait recommander des films similaires en fonction de leurs genres, acteurs ou réalisateurs.

- **Systèmes de recommandation hybrides (Hybrid Recommender Systems)** : Ces systèmes combinent plusieurs modèles de recommandation pour améliorer la qualité des recommandations. Par exemple, un système de recommandation hybride pourrait combiner le filtrage collaboratif avec un modèle basé sur le contenu pour recommander des éléments similaires à ceux que l'utilisateur a aimé et qui ont également été appréciés par d'autres utilisateurs similaires.


Pour evaluer les modèles, je vais utiliser 3 metriques :

- **La précision** est une mesure qui évalue la proportion d'articles recommandés qui sont pertinents pour un utilisateur donné. Plus précisément, elle correspond au nombre d'articles pertinents recommandés divisé par le nombre total d'articles recommandés.

- **Le Mean Average Precision *(MAP)*** est une mesure qui évalue la qualité du classement des articles recommandés. Il calcule la précision moyenne sur toutes les positions dans la liste des recommandations. Il prend également en compte les articles pertinents qui ne sont pas recommandés en les incluant dans le calcul de la précision moyenne.

- **Le Normalized Discounted Cumulative Gain *(NDCG)*** est une mesure qui évalue la qualité du classement des articles recommandés en prenant en compte l'ordre de la recommandation. Il tient compte de la pertinence de chaque article recommandé et de sa position dans la liste de recommandation.

Ces trois métriques sont souvent utilisées pour évaluer la qualité d'un système de recommandation. La précision et le MAP évaluent la pertinence des recommandations tandis que le NDCG prend également en compte l'ordre de recommandation.

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import os
import pickle
from time import time
from random import randint
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, ndcg_at_k

import ipywidgets as widgets

## <center><font color=darkRed>1. - Préparation des données</font></center>

### <center><font color=darkBlue>1.1 - Chargement des données</font></center>

In [2]:
# Root path of the dataset
src_path = Path('../data/news-portal-user-interactions-by-globocom')

# Path to the clicks files
#click_path = Path('../data/clicks')

#df_clicks_sample = pd.read_csv(src_path / 'clicks_sample.csv')
articles = pd.read_csv(src_path / 'articles_metadata.csv')
clicks = pd.read_csv('../data/clicks.csv')
df_final = pd.read_csv('../data/df_final.csv')
# Load pickle data
with open(os.path.join(src_path,'articles_embeddings.pickle'), 'rb') as file:
    embeddings = pickle.load(file)

In [3]:
print('Articles Dataframe shape: ', articles.shape)
print('Embedding Matrix shape: ', embeddings.shape)
print('Clicks Dataframe shape: ', clicks.shape)

Articles Dataframe shape:  (364047, 5)
Embedding Matrix shape:  (364047, 250)
Clicks Dataframe shape:  (2988181, 13)


In [4]:
print('Clicks Dataframe shape: ', df_final.shape)
print("Nombre unique d'utilisateur: ", df_final.user_id.nunique())
print("Nombre d'article unique: ", articles.article_id.nunique())
print("Nombre d'article lu par les utilisateurs: ", df_final.article_id.nunique())

Clicks Dataframe shape:  (1245785, 29)
Nombre unique d'utilisateur:  28891
Nombre d'article unique:  364047
Nombre d'article lu par les utilisateurs:  25503


### <center><font color=darkBlue>1.2 - Préparation des données</font></center>

In [5]:
df_score = df_final[['user_id', 'article_id','interest_score']]
df_score.head()

,user_id,article_id,interest_score
0,44,157541,2.0
1,121,157541,2.0
2,143,157541,2.0
3,153,157541,2.0
4,206,157541,2.0


In [6]:
RATIO = 0.10
df_score_sample = df_score.sample(frac=RATIO, random_state=8989) 
print(df_score_sample.shape)
df_score_sample.head()

(124578, 3)


,user_id,article_id,interest_score
561,6972,68866,2.0
713439,66899,58565,2.0
85653,65976,160974,2.0
183461,52354,207024,4.0
253342,19662,123909,2.0


## <center><font color=darkRed>2. - Implémentation, entrainement et comparaison des modèles </font></center>

### <center><font color=darkBlue>2.1 - Librairie Surprise</font></center>

**Surprise (Simple Python RecommendatIon System Engine)** est une bibliothèque open-source de **filtrage collaboratif *(Collaborative Filtering)*** axée sur les évaluations explicites des utilisateurs, comme les notes ou les critiques, pour recommander des produits. 

- **KNNBasic** est un modèle de filtrage collaboratif basé sur les k-plus proches voisins. Il utilise la similarité cosinus entre les utilisateurs ou les éléments pour prédire les notes. Il s'agit d'un modèle simple et efficace pour les systèmes de recommandation avec peu de données.

- **SVD *(Singular Value Decomposition)*** est un modèle de factorisation de matrice largement utilisé pour la recommandation. Il décompose une matrice utilisateur-article en deux matrices de faible rang, une pour les utilisateurs et l'autre pour les articles. Cette décomposition permet de trouver des relations cachées entre les utilisateurs et les articles et de prédire les notes manquantes.

- **BaselineOnly** est un modèle qui prédit les notes en utilisant une simple ligne de base. Il estime la note moyenne globale, ainsi que les biais utilisateur et article. Ces biais représentent les préférences individuelles des utilisateurs et des articles. En utilisant ces estimations, BaselineOnly peut prédire les notes manquantes avec précision.

- **CoClustering** est un modèle de regroupement coopératif qui regroupe les utilisateurs et les articles simultanément. Il estime les notes en fonction des clusters auxquels appartiennent les utilisateurs et les articles. Ce modèle est utile pour les ensembles de données très dispersés.

Ces modèles sont tous des modèles de **filtrage collaboratif** qui utilisent des approches différentes pour prédire les notes manquantes dans une matrice **utilisateur-article** *(Cette matrice contient des notes ou des évaluations pour chaque utilisateur et chaque article dans l'ensemble de données. Cependant, il est courant que certaines entrées soient manquantes dans cette matrice car les utilisateurs n'ont pas évalué tous les articles. L'objectif du filtrage collaboratif est de prédire ces notes manquantes en utilisant les données disponibles).*



In [33]:
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import KNNBasic
from surprise import CoClustering
from surprise import BaselineOnly
from surprise import accuracy
from surprise.model_selection import train_test_split

def train_and_compare_models(data):
    """
    Cette fonction entraîne plusieurs modèles de recommandation sur un ensemble de données et compare leurs performances
    en termes de RMSE, MSE et MAE. Les modèles évalués sont : Collaborative Filtering, Matrix Factorization, Content-Based,
    et Hybrid Recommender Systems.

    Arguments:
    - data (pandas.DataFrame) : un DataFrame pandas contenant les notes d'intérêt des utilisateurs pour les articles.

    Retourne:
    - resultas_table (pandas.DataFrame) : Un DataFrame pandas contenant les performances de chaque modèle sur l'ensemble de test en termes de RMSE, MSE et MAE.
    """

    # Définir la plage de notes (min, max) dans les données
    reader = Reader(rating_scale=(1, 10))

    # Convertir les données en un objet Surprise Dataset
    dataset = Dataset.load_from_df(data[['user_id', 'article_id', 'interest_score']], reader)

    # Diviser les données en ensembles d'entraînement et de test
    trainset, testset = train_test_split(dataset, test_size=0.25)

    # Définir les différents modèles à entraîner
    models = [
        ('Collaborative Filtering', KNNBasic(sim_options={'user_based': True})),
        ('Matrix Factorization', SVD(n_factors=128, biased=False)),
        ('Content-Based', BaselineOnly()),
        ('Hybrid Recommender Systems', CoClustering())
    ]

    # Entraîner chaque modèle sur l'ensemble d'entraînement et obtenir des prédictions pour l'ensemble de test
    results = []
    for name, model in models:
        model.fit(trainset)
        predictions = model.test(testset)
        rmse = accuracy.rmse(predictions, verbose=False)
        mse = accuracy.mse(predictions, verbose=False)
        mae = accuracy.mae(predictions, verbose=False)
        results.append((name, rmse, mse, mae))

    # Stocker les résultats dans un tableau et le renvoyer en sortie de la fonction
    results_table = pd.DataFrame(results, columns=['Model', 'RMSE','MSE', 'MAE'])
    return results_table



In [34]:
# Entraîner et comparer différents modèles de recommandation
results_table = train_and_compare_models(df_score_sample)

# Afficher les résultats dans la console
print(results_table)

Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


/Users/charlenehourdin/opt/anaconda3/envs/OC-P9/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


                        Model      RMSE       MSE       MAE
0     Collaborative Filtering  0.867708  0.752917  0.675909
1        Matrix Factorization  1.824735  3.329658  1.606351
2               Content-Based  0.751193  0.564291  0.616864
3  Hybrid Recommender Systems  0.864043  0.746570  0.676196


#### <center><font color=darkGreen>2.1.1 - Choix du meilleur modèle et prédiction</font></center>

In [35]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

def training_reco_SVD(data, user_id, n_reco=5, train=True, model_path=None):
    """
    Cette fonction utilise la factorisation de matrice (SVD) pour recommander des articles à un utilisateur spécifique.
    
    Arguments:
    - data (pandas.DataFrame): dataframe contenant les données d'intérêt des utilisateurs pour les articles.
    - user_id (int): l'identifiant de l'utilisateur pour lequel nous voulons faire des recommandations.
    - n_reco (int): le nombre d'articles à recommander à l'utilisateur (par défaut: 5).
    - train (pandas.DataFrame): booléen indiquant s'il faut entraîner le modèle ou non (par défaut: True).
    - model_path (str): chemin vers un fichier contenant un modèle déjà entraîné. Si spécifié, le modèle est chargé à partir de ce chemin (par défaut: None).
    
    Retourne:
    - rec_df (pandas.DataFrame) : Un dataframe contenant les n_reco articles recommandés à l'utilisateur avec leur score de recommandation.
    """

    # Créer un jeu de données surprise
    reader = Reader(rating_scale=(0, 1))
    dataset = Dataset.load_from_df(data[['user_id', 'article_id', 'interest_score']], reader)

    # Diviser l'ensemble de données en ensembles d'entraînement et de test
    trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)

    # Entrainer le modèle si necessaire
    if train or model_path is None:
        model = SVD(n_factors=128, random_state=42)
        print("[INFO] : Start training model")
        model.fit(trainset)
        
        # sauvegarder le modèle si necessaire
        with open('../model/recommender.SVD', 'wb') as filehandle:
            pickle.dump(model, filehandle)
    else:
        with open('../model/recommender.SVD', 'rb') as filehandle:
            model = pickle.load(filehandle)

    # Obtenir les recommendations
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    recommendations = model.test(testset)
    recommendations = [(int(x.iid), x.est) for x in recommendations]

    # Créer une dataframe avec les articles recommandés et leurs scores
    rec_df = pd.DataFrame(recommendations, columns=['article_id', 'score'])

    # Trier par score et sélectionner les meilleures recommandations
    rec_df = rec_df.sort_values(by='score', ascending=False).head(n_reco)

    return rec_df


In [36]:
recs = training_reco_SVD(df_score_sample, user_id=44, n_reco=5, train=True, model_path=None)

# afficher les recommandations
print(recs)

[INFO] : Start training model
      article_id  score
0         234267      1
6081      169171      1
6075      348084      1
6076      106825      1
6077      363074      1


### <center><font color=darkBlue>2.2 - Librairie Implicit</font></center>

La bibliothèque **Implicit** est utilisée pour la recommandation basée sur les préférences implicites des utilisateurs. Contrairement à la bibliothèque Surprise qui se concentre sur les données explicites *(notes ou évaluations)*, **Implicit** se concentre sur les interactions implicites telles que les les clics, les likes, les commentaires et les lectures.

Nous allons tester les 3 modèles suivant : 

- **AlternatingLeastSquares *(ALS)***  est un modèle de factorisation de matrice largement utilisé pour la recommandation basée sur les préférences implicites des utilisateurs. Il décompose une matrice utilisateur-article en deux matrices de faible rang, une pour les utilisateurs et l'autre pour les articles. Cette décomposition permet de trouver des relations cachées entre les utilisateurs et les articles et de prédire les articles que les utilisateurs pourraient aimer en fonction de leurs interactions implicites.

- **BayesianPersonalizedRanking *(BPR)*** est un modèle de classement qui utilise les interactions implicites des utilisateurs pour apprendre des préférences individuelles. Il utilise une méthode de gradient stochastique pour estimer les préférences des utilisateurs pour les articles en maximisant la probabilité que l'utilisateur préfère l'article qu'il a interagi plutôt que les autres articles.

- **LogisticMatrixFactorization *(LMF)*** est un modèle qui utilise la régression logistique pour modéliser les préférences des utilisateurs pour les articles en fonction de leurs interactions implicites. Il utilise une approche de factorisation de matrice pour estimer les caractéristiques cachées des utilisateurs et des articles, puis utilise ces caractéristiques pour prédire les articles que les utilisateurs pourraient aimer.


#### <center><font color=darkGreen>2.2.1 - Séparation du jeu de donnée (Train, test, split)</font></center>

In [37]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_score_sample, train_size=0.8, random_state=41)

#### <center><font color=darkGreen>2.2.2 - Modèles sans matrice d'embeddings</font></center>

In [40]:
def train_eval_models(train_df, test_df, n_recs, reg_param=0.01, factors=0):
    """
    Cette fonction prend en entrée les données d'entraînement et de test sous forme de dataframes et des paramètres pour
    l'entraînement des modèles. Elle entraîne plusieurs modèles de recommandation (AlternatingLeastSquares, 
    BayesianPersonalizedRanking et LogisticMatrixFactorization) et calcule les mesures de performance (précision, MAP 
    et NDCG) pour chaque modèle.
    
    Arguments:
    
    - train_df (pandas.DataFrame): dataframe des données d'entraînement contenant les colonnes 'user_id', 'article_id' et 'interest_score'
    - test_df (pandas.DataFrame): dataframe des données de test contenant les colonnes 'user_id', 'article_id' et 'interest_score'
    - n_recs (int): le nombre de recommandations à générer pour chaque utilisateur
    - reg_param (float, optional): le paramètre de régularisation pour la régularisation L2 des facteurs latents (par défaut: 0.01).
    - factors (int, optional): le nombre de facteurs latents à utiliser pour la factorisation matricielle (par défaut: 0).
    
    Retourne:
    - df_results (pandas.DataFrame) : un dataframe contenant les résultats d'évaluation des modèles.
    """
    
    df_results = pd.DataFrame(columns=['model', 'precision', 'map', 'ndcg', 'train_time'])
    
    dim = (max(train_df.user_id.max(), test_df.user_id.max()) + 1,
           max(train_df.article_id.max(), test_df.article_id.max()) + 1)
    
    train_csr = csr_matrix((train_df['interest_score'], (train_df['user_id'], train_df['article_id'])), dim)
    test_csr = csr_matrix((test_df['interest_score'], (test_df['user_id'], test_df['article_id'])), dim)
    
    models = [AlternatingLeastSquares(regularization=reg_param),
              BayesianPersonalizedRanking(regularization=reg_param),
              LogisticMatrixFactorization(factors=factors, regularization=reg_param, random_state=42)]
    
    for model in models:
        
        print("##" * 30)
        print("[INFO] : Commencer l'entraînement du modèle: ", model.__class__.__name__)
        
        # Launch the timer
        train_start_time = time()
        
        model.fit(train_csr)
        
        # Stop the timer and calculate the training time
        train_time = time() - train_start_time
        
        # Calculate evaluation metrics
        precision = round(precision_at_k(model, train_csr, test_csr, K=n_recs), 5)
        map_score = round(mean_average_precision_at_k(model, train_csr, test_csr, K=n_recs), 5)
        ndcg = round(ndcg_at_k(model, train_csr, test_csr, K=n_recs), 5)
        
        # Log results in the results dataframe
        df_results = df_results.append({
            'model': model.__class__.__name__,
            'precision': precision,
            'map': map_score,
            'ndcg': ndcg,
            'train_time': round(train_time, 2),
        }, ignore_index=True)
        
    return df_results


In [51]:
results = train_eval_models(train_df, test_df, n_recs=5, reg_param=0.05, factors=0)
# Print results
print(results)

############################################################
[INFO] : Commencer l'entraînement du modèle:  AlternatingLeastSquares


100%|██████████| 15377/15377 [00:24<00:00, 617.84it/s]


############################################################
[INFO] : Commencer l'entraînement du modèle:  BayesianPersonalizedRanking


100%|██████████| 15377/15377 [00:27<00:00, 564.51it/s]


############################################################
[INFO] : Commencer l'entraînement du modèle:  LogisticMatrixFactorization


100%|██████████| 15377/15377 [00:25<00:00, 593.88it/s]

                         model  precision      map     ndcg  train_time
0      AlternatingLeastSquares    0.00628  0.00349  0.00470        9.86
1  BayesianPersonalizedRanking    0.00024  0.00020  0.00023        6.09
2  LogisticMatrixFactorization    0.01726  0.00887  0.01231        0.52


#### <center><font color=darkGreen>2.2.3 - Modèles avec matrice d'embeddings</font></center>

La matrice d'embedding sert à représenter les utilisateurs et les articles sous forme de vecteurs dans un espace de dimension réduite.

L'objectif de la matrice d'embedding est de capturer les relations entre les utilisateurs et les articles en projetant ces derniers dans un espace latent où les distances entre les vecteurs représentent la similitude entre les utilisateurs et les articles. En effet, dans un espace de dimension réduite, il est plus facile de trouver des similitudes entre les utilisateurs et les articles.

In [49]:
def train_eval_models_emb(train_df, test_df, n_recs, reg_param=0.01, factors=0, emb_matrix=None):
    """
    Cette fonction entraîne trois modèles de recommandation: AlternatingLeastSquares, BayesianPersonalizedRanking et 
    LogisticMatrixFactorization et évalue leurs performances à l'aide de différentes métriques telles que la précision,
    la moyenne de précision, et NDCG. Si une matrice d'embedding est fournie, les modèles peuvent utiliser ces informations
    pour améliorer la qualité des recommandations.

    Argument:
    - train_df (pandas.DataFrame): dataframe contenant les données d'entraînement pour les modèles de recommandation.
    - test_df (pandas.DataFrame): dataframe contenant les données de test pour évaluer la performance des modèles.
    - n_recs (int): nombre de recommandations à générer pour chaque utilisateur.
    - reg_param (float, optional): paramètre de régularisation pour les modèles ALS, BPR et LMF. (par défaut: 0.01).
    - factors (int, optional): nombre de facteurs à utiliser pour le modèle LMF. (par défaut: 0)..
    - emb_matrix (numpy.ndarray, optional): matrice d'embedding pour les articles. (par défaut: None).

    Retourne:
    - df_results (pandas.DataFrame): Un dataframe contenant les résultats d'évaluation des modèles.
    """
    df_results = pd.DataFrame(columns=['model', 'precision', 'map', 'ndcg', 'train_time'])
    
    dim = (max(train_df.user_id.max(), test_df.user_id.max()) + 1,
           max(train_df.article_id.max(), test_df.article_id.max()) + 1)
    
    train_csr = csr_matrix((train_df['interest_score'], (train_df['user_id'], train_df['article_id'])), dim)
    test_csr = csr_matrix((test_df['interest_score'], (test_df['user_id'], test_df['article_id'])), dim)
    
    als_model = AlternatingLeastSquares(regularization=reg_param)
    bpr_model = BayesianPersonalizedRanking(regularization=reg_param)
    lmf_model = LogisticMatrixFactorization(factors=factors, regularization=reg_param)

    for model in [als_model, bpr_model, lmf_model]:
        
        print("##" * 30)
        print("[INFO] : Commencer l'entraînement du modèle: ", model.__class__.__name__)
        
        # Lancer le timer
        train_start_time = time()
        
        # Vérifier si les intégrations sont fournies
        if emb_matrix is not None:
            # remplace les facteurs d'articles du modèle par la matrice d'embedding, 
            # ce qui permet au modèle de tirer parti des informations d'embedding pour l'entraînement. 
            if model == AlternatingLeastSquares or model == LogisticMatrixFactorization:
                model.item_factors = emb_matrix
                model.user_factors = np.zeros((train_csr.shape[0], emb_matrix.shape[1]))

            elif model == BayesianPersonalizedRanking:
                model.user_factors = emb_matrix
            
        # Former le modèle choisi
        model.fit(train_csr)
        
       # Arrêtez le chronomètre et calculez le temps d'entraînement
        train_time = time() - train_start_time
        
        
        # Calculer les métriques d'évaluation
        precision = round(precision_at_k(model, train_csr, test_csr, K=n_recs), 5)
        map_score = round(mean_average_precision_at_k(model, train_csr, test_csr, K=n_recs), 5)
        ndcg = round(ndcg_at_k(model, train_csr, test_csr, K=n_recs), 5)
        #recall =  round(recall_at_k(model, train_csr, test_csr, K=n_recs), 5)
        # Consigner les résultats dans la trame de données des résultats
        df_results = df_results.append({
            'model': model.__class__.__name__,
            'precision': precision,
            'map': map_score,
            'ndcg': ndcg,
            #'recall':recall,
            'train_time': round(train_time, 2),
        }, ignore_index=True)
        
    return df_results


In [50]:
results = train_eval_models_emb(train_df, test_df, n_recs = 5, reg_param=0.05, factors=0, emb_matrix=embeddings)
# Print results
print(results)

############################################################
[INFO] : Commencer l'entraînement du modèle:  AlternatingLeastSquares


100%|██████████| 15377/15377 [00:25<00:00, 614.97it/s]


############################################################
[INFO] : Commencer l'entraînement du modèle:  BayesianPersonalizedRanking


100%|██████████| 15377/15377 [00:25<00:00, 606.63it/s]


############################################################
[INFO] : Commencer l'entraînement du modèle:  LogisticMatrixFactorization


100%|██████████| 15377/15377 [00:23<00:00, 657.81it/s]

                         model  precision      map     ndcg  train_time
0      AlternatingLeastSquares    0.00518  0.00284  0.00385        9.23
1  BayesianPersonalizedRanking    0.00033  0.00019  0.00024        6.11
2  LogisticMatrixFactorization    0.02411  0.01220  0.01696        0.53


Les résultats montrent que l'utilisation d'un modèle d'embedding a amélioré la performance du modèle **LogisticMatrixFactorization** en termes de **précision**, **map** et **ndcg**, tandis que les performances de **AlternatingLeastSquares** et **BayesianPersonalizedRanking** ont légèrement diminué. 

Cependant, les différences entre les deux ensembles de résultats ne sont pas très importantes, il est donc possible que les améliorations apportées par l'utilisation d'un modèle d'embedding ne soient pas significatives dans ce cas spécifique. Par ailleurs, les temps d'entraînement des modèles sont restés relativement similaires.


#### <center><font color=darkGreen>2.2.4 - Choix du meilleur modèle et prédiction</font></center>

Pour le choix du meilleur modèle, je vais utiliser celui sans la matrice d'embedding. En effet la matrice d'embedding a améliorer très faiblement la performance du modèle et le temps d'entraînement à légèrement augmenter, afin de ne pas alourdir mon API, je préfère utiliser le modèle **LogisticMatrixFactorization** sans embeddings.

En se basant uniquement sur les mesures de performance présentées dans le tableau de comparaison, le modèle **LogisticMatrixFactorization** est le meilleur en termes de précision **(17%)**. Cela signifie que 17% des articles recommandés par le système sont des articles pertinents pour l'utilisateur, donc susceptibles de lui plaire.

In [14]:
import pandas as pd
from scipy.sparse import csr_matrix

def training_reco_LMF(data, user_id, n_reco=5,factors=0, reg_param=0.01, train=True, model_path=None):
    """
    Entraîne un modèle de recommandation basé sur la factorisation de matrice.
    Utilise une instance de LogisticMatrixFactorization.

    Arguments:
        - data (pandas.DataFrame): dataframe contenant les données d'entraînement pour les modèles de recommandation.
        - user_id (int): L'ID de l'utilisateur pour lequel obtenir des recommandations.
        - n_reco (int): Le nombre de recommandations à générer (par défaut 5).
        - factors (int): Le nombre de facteurs de latent à utiliser (par défaut 0).
        - reg_param (float): Le paramètre de régularisation à utiliser (par défaut 0.01).
        - train (bool): Indique si le modèle doit être entraîné (par défaut True).
        - model_path (str): Le chemin où enregistrer le modèle entraîné (par défaut None).

    Retourne:
        - rec_df (pandas.DataFrame): Un dataframe contenant les articles recommandés et leurs scores.
    """
    # compute interaction matrix
    interactions = data.groupby(['user_id', 'article_id']).size().reset_index(name='count')
    csr_item_user = csr_matrix((interactions['count'].astype(float),
                                (interactions['article_id'], interactions['user_id'])))
    csr_user_item = csr_matrix((interactions['count'].astype(float),
                                (interactions['user_id'], interactions['article_id'])))

    # former le modèle si nécessaire
    if train or model_path is None:
        model = lmf_model = LogisticMatrixFactorization(factors=factors, regularization=reg_param)
        print("[INFO] : Start training model")
        model.fit(csr_user_item)

        # enregistrer le modèle sur le disque si nécessaire
        with open('../model/recommender.LMF.pickle', 'wb') as filehandle:
            pickle.dump(model, filehandle)
    #else:
        #with open('../model/recommender.LMF', 'rb') as filehandle:
            #model = pickle.load(filehandle)

    # obtenir des recommandations
    recommendations = model.recommend(user_id, csr_user_item[user_id], N=n_reco, filter_already_liked_items=True)
    #item_ids = [elt[0] for elt in recommendations]

    # créer un dataframe avec les éléments recommandés et leurs scores
    rec_df = pd.DataFrame({'article_id': recommendations[0], 'score': recommendations[1]})

    # join with article metadata to get article titles and authors
    #rec_df = rec_df.merge(clicks[['article_id']].drop_duplicates(), on='article_id', how='left')

    # trier par score et sélectionner les meilleures recommandations
    #rec_df = rec_df.sort_values(by='score', ascending=False).head(n_reco)

    return rec_df


In [15]:
recs = training_reco_LMF(df_score_sample, user_id=44, n_reco=5,factors=128, reg_param=0.05, train=True, model_path=None)

# afficher les recommandations
print(recs)

[INFO] : Start training model


100%|██████████| 30/30 [00:33<00:00,  1.13s/it]


   article_id     score
0      336223  6.593441
1       96074  6.184423
2      285719  4.814849
3       96470  4.698869
4      235230  3.945615


In [16]:
with open('../model/recommender.LMF.pickle', 'rb') as handle:
    model = pickle.load(handle)
    
def recommend_articles_LMF(data, user_id, n_reco=5):
    """
    Cette fonction prend en entrée un jeu de données de lecture d'articles, l'identifiant d'un utilisateur et un nombre
    d'articles à recommander. Elle calcule la matrice d'interaction utilisateur-article, utilise un modèle de factorisation
    matricielle pour prédire les articles les plus susceptibles d'être lus par l'utilisateur, et renvoie un dataframe
    contenant les identifiants des articles recommandés et leur score de pertinence.
    
    Arguments :
    - data (pandas.DataFrame): dataframe contenant les données le modèle de recommandation.
    - user_id (int): identifiant de l'utilisateur pour lequel on souhaite obtenir des recommandations.
    - n_reco (int): nombre d'articles à recommander. (par défaut: 5).
    
    Retourne :
    - rec_df : Un dataframe contenant les identifiants des articles recommandés et leur score de pertinence.
    """
    # calculer la matrice d'interaction
    interactions = data.groupby(['user_id', 'article_id']).size().reset_index(name='count')
    csr_item_user = csr_matrix((interactions['count'].astype(float),
                                (interactions['article_id'], interactions['user_id'])))
    csr_user_item = csr_matrix((interactions['count'].astype(float),
                                (interactions['user_id'], interactions['article_id'])))


    # obtenir des recommandations
    recommendations = model.recommend(user_id, csr_user_item[user_id], N=n_reco, filter_already_liked_items=True)
    #item_ids = [elt[0] for elt in recommendations]

    # créer un dataframe avec les éléments recommandés et leurs scores
    rec_df = pd.DataFrame({'article_id': recommendations[0], 'score': recommendations[1]})
    
    rec_df['score'] = rec_df['score'].round(2)
    # join with article metadata to get article titles and authors
    #rec_df = rec_df.merge(clicks[['article_id']].drop_duplicates(), on='article_id', how='left')

    # trier par score et sélectionner les meilleures recommandations
    #rec_df = rec_df.sort_values(by='score', ascending=False).head(n_reco)

    return rec_df

In [17]:
recs = recommend_articles_LMF(clicks, user_id=44, n_reco=5)

# afficher les recommandations
print(recs)

   article_id  score
0      336223   6.59
1       96074   6.18
2      285719   4.81
3       96470   4.70
4      235230   3.95


In [18]:
import gzip
with open('../model/recommender.LMF.pickle', 'rb') as handle:
    model = pickle.load(handle)
# Pickle et compression en gzip
with gzip.open('../model/recommender.LMF.pickle.gz', 'wb') as f:
    pickle.dump(model, f)

In [19]:
# Lecture du fichier compressé
#with gzip.open('../model/recommender.LMF.pickle.gz', 'rb') as f:
#   data_loaded = pickle.load(f)

In [20]:
#import urllib
#url = 'https://mycontentstockage.blob.core.windows.net/recop9/recommender.LMF.pickle.gz'
#with urllib.request.urlopen(url) as response:
#    with gzip.GzipFile(fileobj=response) as uncompressed:
#        model = pickle.load(uncompressed)

### <center><font color=darkBlue>2.2 - Modèle de recommandation de popularité</font></center>

In [19]:
def get_popularity_rec(user_id, clicks, n_reco=5):
    """
    Cette fonction prend en entrée l'ID de l'utilisateur et un DataFrame contenant les données de clics.
    Elle utilise la popularité des articles pour recommander les articles les plus populaires qui n'ont pas encore été vus par l'utilisateur.
    
    Arguments :
    - clicks (pandas.DataFrame): DataFrame contenant les données d'entraînement
    - user_id (int): identifiant de l'utilisateur pour lequel on souhaite obtenir des recommandations.
    - n_reco (int): nombre d'articles à recommander.(par défaut: 5).
    
    Retourne :
    - recommendations_df : Un dataframe contenant les identifiants des articles recommandés et leur score de pertinence.
    """
    print("l'ID utilisateur est : ", user_id)
    # Obtenir les articles que l'utilisateur a déjà vus
    viewed_articles = clicks[clicks['user_id'] == user_id]['article_id'].unique()
    
    # Obtenir les articles les plus populaires
    df_popularity = clicks.groupby(by=['article_id'])['click_timestamp'].count().sort_values(ascending=False).reset_index()
    df_popularity.rename(columns={'click_timestamp': 'popularity'}, inplace=True)
    df_popularity = df_popularity[~df_popularity['article_id'].isin(viewed_articles)]
    df_popularity['score'] = df_popularity['popularity'] / df_popularity['popularity'].max()
    
    print('Les articles recommandés sont: ')
    # Obtenir les n_reco articles les plus populaires
    reco = df_popularity[['article_id', 'score']].head(n_reco).to_dict('records')

    # Créer un DataFrame pour stocker les recommandations
    recommendations_df = pd.DataFrame(reco)

    return recommendations_df


In [20]:
results = get_popularity_rec(44, clicks, n_reco=5)
print(results)

l'ID utilisateur est :  44
Les articles recommandés sont: 
   article_id     score
0      272143  1.000000
1      336221  0.824068
2      234698  0.811906
3      123909  0.798881
4      336223  0.755105


### <center><font color=darkBlue>2.3 - Modèle de recommandation Content based</font></center>

In [29]:
def get_cb_reco(user_id, clicks, embeddings, n_reco=5):
    
    """
    Cette fonction retourne les recommandations basées sur le contenu en utilisant les embeddings et la similarité cosinus.
    
    Args:
    - user_id (int): ID de l'utilisateur pour lequel nous voulons obtenir des recommandations
    - clicks (pandas.DataFrame): DataFrame contenant les interactions utilisateur-article
    - embeddings (numpy.ndarray, optional): matrice d'embedding pour les articles.  Matrice d'embeddings pour les articles
    - n_reco (int): Nombre d'articles à recommander (par défaut 5)
    
    Returns:
    - recommendations_df (pandas.DataFrame): DataFrame contenant les recommandations pour l'utilisateur avec leur score de similarité cosinus
    """

    print("l'ID utilisateur est : ", user_id)
    
    # identifier le dernier article lu par l'utilisateur
    var = clicks.loc[clicks.user_id == user_id]['article_id'].to_list()
    value = var[-1]
    print("Le dernier article lu par l'utilisateur est: ", value)
    
    # Suppression de tous les embeddings sauf celui correspondant à l'article le plus récemment cliqué
    emb = embeddings
    for i in range(0, len(var)):
        if i != value:
            emb = np.delete(emb, [i], 0)

    # Suppression de l'embedding correspondant à l'article le plus récemment cliqué
    temp = np.delete(emb, [value], 0)

    # Calcul de la similarité cosinus entre l'article le plus récemment cliqué et les autres articles
    distances = cosine_similarity([emb[value]], temp)[0]
    
    # Tri des articles recommandés en fonction de leur similarité cosinus
    ranked_ids = np.argsort(distances)[::-1][0:n_reco]
    ranked_similarities = np.sort(distances)[::-1][0:n_reco]
    print('Les articles recommandés sont: ')
    
    # créer une liste de dictionnaires pour stocker les recommandations et leurs scores
    recommendations = []
    for i in range(len(ranked_ids)):
        recommendation = {}
        #recommendation['user_id'] = userID
        recommendation['article_id'] = ranked_ids[i]
        recommendation['score'] = ranked_similarities[i]
        recommendations.append(recommendation)
        
    
    # créer un DataFrame à partir de la liste des dictionnaires
    recommendations_df = pd.DataFrame(recommendations)
    
    return recommendations_df



In [30]:
results = get_cb_reco(44, clicks, embeddings, n_reco=5)
print(results)

l'ID utilisateur est :  44
Le dernier article lu par l'utilisateur est:  88914
Les articles recommandés sont: 
   article_id     score
0       89554  0.928836
1       89128  0.925168
2       89713  0.921694
3       89507  0.915548
4       89765  0.913209


In [27]:
def collaborative_filtering(user_id, clicks, n_reco=5):
    """
    Cette fonction utilise le filtrage collaboratif pour recommander des articles à un utilisateur en fonction de ses interactions
    passées avec les articles.
    
    Arguments:
        - clicks (pandas.DataFrame): DataFrame contenant les données d'entraînement
        - user_id (int): identifiant de l'utilisateur pour lequel on souhaite obtenir des recommandations.
        - n_reco (int): nombre d'articles à recommander.(par défaut: 5).
    
    Returns:
        - recommendations_df (pandas.DataFrame): contenant les articles recommandés et leurs scores
    """
    print("l'ID utilisateur est : ", user_id)
    # Récupérer la liste des articles cliqués par l'utilisateur
    user_clicks = clicks[clicks['user_id'] == user_id]['article_id'].tolist()
    
    # Créer un dataframe contenant uniquement les articles qui ont été cliqués par l'utilisateur
    user_clicks_df = clicks[clicks['article_id'].isin(user_clicks)]
    
    # Créer une matrice d'interaction utilisateur-article
    interactions = pd.crosstab(user_clicks_df['user_id'], user_clicks_df['article_id'])
    
    # Appliquer une factorisation matricielle à la matrice d'interaction
    U, sigma, Vt = np.linalg.svd(interactions, full_matrices=False)
    
    # Récupérer les vecteurs d'utilisateur et d'article en fonction de l'ID utilisateur
    user_index = interactions.index.get_loc(user_id)
    user_vector = U[user_index,:]
    article_vectors = Vt.T
    
    # Calculer les scores de similarité cosinus entre l'utilisateur et les articles
    similarities = cosine_similarity([user_vector], article_vectors)
    similarity_scores = pd.Series(similarities[0], index=interactions.columns)
    
    # Récupérer les n articles les plus similaires
    top_articles = similarity_scores.sort_values(ascending=False)[:n_reco]
    recommendations = top_articles.index.tolist()
    recommendation_scores = top_articles.values.tolist()
    print('Les articles recommandés sont: ')
    
    # Créer une liste de dictionnaires contenant les recommandations et leurs scores
    recommendation_list = []
    for article, score in zip(recommendations, recommendation_scores):
        recommendation_dict = {'article_id': article, 'score': score}
        recommendation_list.append(recommendation_dict)
        
    # Créer un DataFrame à partir de la liste de dictionnaires
    recommendations_df = pd.DataFrame(recommendation_list)
    
    return recommendations_df


In [28]:
results = collaborative_filtering(44, clicks, n_reco=5)
print(results)

l'ID utilisateur est :  44
Les articles recommandés sont: 
   article_id     score
0       89622  0.700093
1      305136  0.453587
2      114161  0.252705
3      257561  0.197036
4       88915  0.184968
